In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
import datetime as dt
%matplotlib inline

In [2]:
df=pd.read_csv('data/churn_train.csv')

In [3]:
df.head()
df.info()
df['weekday_pct_bins'] = pd.cut(df['weekday_pct']*df['surge_pct'], bins=4, labels=['0-25%','25-50%', '50-75%', '75-100%'])
WeekdayDummies = pd.get_dummies(df['weekday_pct_bins'])


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 12 columns):
avg_dist                  40000 non-null float64
avg_rating_by_driver      39838 non-null float64
avg_rating_of_driver      33472 non-null float64
avg_surge                 40000 non-null float64
city                      40000 non-null object
last_trip_date            40000 non-null object
phone                     39681 non-null object
signup_date               40000 non-null object
surge_pct                 40000 non-null float64
trips_in_first_30_days    40000 non-null int64
luxury_car_user           40000 non-null bool
weekday_pct               40000 non-null float64
dtypes: bool(1), float64(6), int64(1), object(4)
memory usage: 3.4+ MB


In [4]:
df['last_trip_date']=pd.to_datetime(df['last_trip_date'])
df['signup_date']=pd.to_datetime(df['signup_date'])
today=df.last_trip_date.max()

In [5]:
df['Churn']= df['last_trip_date'].apply(lambda x : 1 if today-x> timedelta(days=30) else 0)

In [6]:
df['daysSinceSignup']=today-timedelta(days=30)-df['signup_date']
df['daysSinceSignup']=df['daysSinceSignup'].astype('timedelta64[D]').astype(int)

dummies=pd.get_dummies(df.city)
df=pd.concat([df, dummies], axis=1)
df=pd.concat([df,WeekdayDummies], axis=1)



In [7]:
df.drop(['city','signup_date','last_trip_date'],inplace=True,axis=1)
df=df.dropna()

In [8]:
df['luxury_car_user']=df['luxury_car_user'].apply(lambda x: 1 if x==True else 0)

In [9]:
df['phone']=df['phone'].apply(lambda x: 1 if x=='iPhone' else 0)
df['SurgxDist']=df['avg_dist']*df['avg_surge']

In [10]:

df['IphoneLuxury'] = (df['phone'] > 0) & (df['luxury_car_user'] > 0)
df['IphoneLuxury']=df['IphoneLuxury'].apply(lambda x: 1 if x==True else 0)




In [35]:
X=df[['avg_rating_by_driver','avg_dist','avg_surge',"King's Landing",'0-25%','25-50%','50-75%','75-100%','phone','luxury_car_user']]


In [36]:
y=df['Churn']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)

In [37]:
gbModel=GradientBoostingClassifier(n_estimators=200)
rf=RandomForestClassifier()
from pandas.plotting import scatter_matrix

In [38]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23192 entries, 30322 to 32990
Data columns (total 10 columns):
avg_rating_by_driver    23192 non-null float64
avg_dist                23192 non-null float64
avg_surge               23192 non-null float64
King's Landing          23192 non-null uint8
0-25%                   23192 non-null uint8
25-50%                  23192 non-null uint8
50-75%                  23192 non-null uint8
75-100%                 23192 non-null uint8
phone                   23192 non-null int64
luxury_car_user         23192 non-null int64
dtypes: float64(3), int64(2), uint8(5)
memory usage: 1.2 MB


In [39]:
gbModel.fit(X_train,y_train)
rf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [40]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23192 entries, 30322 to 32990
Data columns (total 10 columns):
avg_rating_by_driver    23192 non-null float64
avg_dist                23192 non-null float64
avg_surge               23192 non-null float64
King's Landing          23192 non-null uint8
0-25%                   23192 non-null uint8
25-50%                  23192 non-null uint8
50-75%                  23192 non-null uint8
75-100%                 23192 non-null uint8
phone                   23192 non-null int64
luxury_car_user         23192 non-null int64
dtypes: float64(3), int64(2), uint8(5)
memory usage: 1.2 MB


In [41]:
gbModel.score(X_test,y_test)
rf.score(X_test,y_test)

0.69999999999999996

In [42]:
gbModel.score(X_test,y_test)

0.75754527162977869